In [11]:
import pandas as pd
import numpy as np
import datetime
import calendar
import math
import geopandas as gpd
import folium
import psutil
import random
import time
def READHDR(link):
    f = open(link, "r", encoding="utf-8")
    header = pd.DataFrame({'col1': f.read().splitlines()})
    f.close()
    header['value'] = header['col1'].apply(lambda x: x.split()[1])
    header['col1'] = header['col1'].apply(lambda x: x.split()[0])
    index = header.col1.unique()
    header.drop(columns='col1', inplace=True)
    header.set_index(index, inplace=True)
    return header

def DF_UTIL(header,city):
    ulxmap = pd.to_numeric(header.iloc[5])
    ulymap = pd.to_numeric(header.iloc[6])
    xdim = pd.to_numeric(header.iloc[7])
    ydim = pd.to_numeric(header.iloc[8])
    lonLatBbox = city.coordinates
    minj = math.floor((lonLatBbox[0] - ulxmap + xdim / 2) / xdim)
    maxi = math.ceil((ulymap - lonLatBbox[1] + ydim / 2) / ydim)

    maxj = math.ceil((lonLatBbox[2] - ulxmap + xdim / 2) / xdim)
    mini = math.floor((ulymap - lonLatBbox[3] + ydim / 2) / ydim)

    nrows = maxi - mini
    ncols = maxj - minj
    return pd.DataFrame(np.array([minj, mini, nrows,ncols]), columns=['valori'], index =['minj', 'mini','nrows', 'ncols'])


# def BILREADER(filename, header):
#     ncols = pd.to_numeric(header.loc['NCOLS', 'value'])
#     nrows = pd.to_numeric(header.loc['NROWS', 'value'])
#     cellecentocinquanta = pd.DataFrame(columns=np.arange(ncols))
#     cont = 0
#     for i in range(nrows):
#         riga = pd.Series(np.fromfile(filename, dtype='<u2', count=ncols, sep="", offset=cont * ncols * 2))
#         cont = cont + 1
#         cellecentocinquanta = cellecentocinquanta.append(riga, ignore_index=True)
#     return cellecentocinquanta

def BILREADER_CORRETTO(filename, header,cnt):
    ncols = pd.to_numeric(header.loc['NCOLS', 'value'])
    nrows = pd.to_numeric(header.loc['NROWS', 'value'])
    cellecentocinquanta = pd.DataFrame(columns=np.arange(ncols))
    for i in range(nrows):
         riga = pd.Series(np.fromfile(filename, dtype='<u2', count=ncols, sep="", offset=i * ncols * 2))
         cellecentocinquanta = cellecentocinquanta.append(riga, ignore_index=True)
    # cellecentocinquanta = cellecentocinquanta.head(50)
    # cellecentocinquanta
    cellecentocinquanta = cellecentocinquanta.reset_index().melt(id_vars= ['index'], value_name= str(cnt))
    cellecentocinquanta['riga-colonna'] = cellecentocinquanta['index'].astype(str) +'-'+cellecentocinquanta.variable.astype(str)
    cellecentocinquanta= cellecentocinquanta[['riga-colonna', str(cnt)]]
    return cellecentocinquanta


class City:
    def __init__(self, name, coordinates):
        self.name = name
        self.coordinates = coordinates


def data_amg(data):
    return data.strftime("%Y%m%d")


def data_amg_om(data):
    return data.strftime("%Y%m%d_%H%M")


# def ToCSV(hb, nrows, ncols, mini, minj, out, start):
#     for i in range(nrows):
#         for j in range(ncols):
#             key = f'{mini + i}-{minj + j}'  #indice di riga e colonna dal minimo mi muovo degli indici
#             v = hb.iloc[(mini + i), (minj + j)]
#             out.write(f'{data_amg_om(start)},{key},{v}\n')
#     return print('fatto')

# def ToCSVConComune(hb, nrows, ncols, mini, minj, out, start, sezionetocomune):
#     for i in range(nrows):
#         for j in range(ncols):
#             sezione = f'{mini + i}-{minj + j}'  #indice di riga e colonna dal minimo mi muovo degli indici
#             valore = hb.iloc[(mini + i), (minj + j)]
#             comune = sezionetocomune.loc[sezione,"comune"]
#             out.write(f'{data_amg_om(start)},{sezione},{comune},{valore}\n')
#     return print('fatto')
#
# def run(area, start, end, mini, minj, nrows, ncols, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare):
#
#         sezionetocomune.comune.fillna( 'Mare', inplace=True)
#         delta = datetime.timedelta(hours= 1)
#         cal = start
#         while cal <= end:
#             time = data_amg(cal)#%Y%m%d
#             giorno = f'{cartella_dei_dati}/{area}/{time}/'    #dove leggiamo
#             outfile = f'{cartella_dove_printare}/{city.name}_{time}_{data_amg(end)}_OK.csv'
#             out = open(outfile, 'a')
#             cnt = 0
#             while cnt < 8:
#                 # hb = pd.DataFrame(np.zeros((pd.to_numeric(header.loc['NROWS', 'value']), pd.to_numeric(header.loc['NCOLS', 'value']))))
#                 hb = BILREADER(f'{giorno}{cnt}.bil', header)
#                 print(f'ho letto {cnt}.bil')
#                 cnt += 1
#                 for f in range(3):
#                     hb = hb + BILREADER(f'{giorno}{cnt}.bil', header)
#                     print(f'ho sommato {cnt}.bil')
#                     cnt += 1
#                 ToCSVConComune(hb/4, nrows, ncols, mini, minj, out, cal, sezionetocomune) #crea la matrice di numeri e la legge q (che sta per quarter)
#                 print(f'ho inserito nel .csv  fino a {cnt}.bil')
#                 cal += delta
#             out.close()
#             break


<>
# def SEZIONI2COMUNI(header, cartella_dove_printare, italia):
#     out = open(f'{cartella_dove_printare}/sezione2comuneITALIA.csv', 'a')
#     out.write(f',comune\n')
#     for i in range(int(header.iloc[3,0])):
#         if i % 200 == 0:
#             print(f'sono alla riga: {i}' )
#         for j in range(int(header.iloc[4,0])):
#             punto = TROVACOORDINATE(f'{i}-{j}',yx,UL) #[y,x]
#             comune = QUADRATINO2COMUNE(gpd.points_from_xy([punto[1]], [punto[0]])[0], italia)
#             # out.write(f'{data_amg_om(start)},{key},{v}\n')
#             out.write(f'{i}-{j},{comune}\n')
#     out.close()
#
# def TROVACOORDINATE(indici, yx, UL):
#     # stringa = indici.replace(' ', '')
#     rigcol = pd.to_numeric(pd.Series(indici.split("-")))
#     delta = yx * rigcol
#     coordinate = [UL[0] - delta[0] - yx[0]/2, UL[1] + delta[1] + yx[1]/2]
#     return coordinate
#
# def QUADRATINO2COMUNE(punto, comuni):
#     comu = ''
#     for i in range(len(comuni.geometry)):
#        if comuni.iloc[i,1].contains(punto):
#             comu = comuni.iloc[i,0]
#             break
#     return comu
def SEZIONI2COMUNI(header, cartella_dove_printare, italia):
    terra = gpd.GeoDataFrame(gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Dati/shp/Mare/Med Land/Med_land.shp')['geometry'])
    out = open(f'{cartella_dove_printare}/sezione2comuneITALIA.csv', 'a')
    out.write(f',comune\n')
    comune = ''

    nrows =np.arange(int(header.iloc[3,0]))
    ncols = np.arange(int(header.iloc[4,0]))
    x = np.array(np.meshgrid(nrows,ncols )).transpose().reshape((int(header.iloc[3,0])*int(header.iloc[4,0]),2))
    inizio = time.time()
    for i in range(len(x)):
        if i % (int(header.iloc[4,0])*5) == 0:
            fine = time.time()
            print(f'checkpoint: {fine-inizio}')
            inizio = time.time()
        punto = TROVACOORDINATE( np.array2string(x[i],separator='-')[1:-1].replace(' ',''),yx,UL) #[y,x]
        comune = QUADRATINO2COMUNE(gpd.points_from_xy([punto[1]], [punto[0]])[0], italia, comune, terra)
        # out.write(f'{data_amg_om(start)},{key},{v}\n')
        # out.write(f'{i}-{j},{comune}\n')

    out.close()


def TROVACOORDINATE(indici, yx, UL):
    # stringa = indici.replace(' ', '')
    rigcol = pd.to_numeric(pd.Series(indici.split("-")))
    delta = yx * rigcol
    coordinate = [UL[0] - delta[0] - yx[0]/2, UL[1] + delta[1] + yx[1]/2]
    return coordinate

def QUADRATINO2COMUNE(punto, comuni, comu, terra):

    if comu in comuni.COMUNE.values:
        if comuni.loc[comuni.COMUNE == comu].iloc[0,1].contains(punto):
            return comu
    else:

        if terra.iloc[0,0].contains(punto):
            for j in range(len(comuni.geometry)):
                if comuni.iloc[j,1].contains(punto):
                    comu = comuni.iloc[j,0]

                    return comu
        else:
            return 'Mare'

def PUNTISEZIONE(sezionetocomune, cartella_dove_printare):
    sezionetocomune.reset_index(inplace= True)
    sezionetocomune.rename( columns={'index':'sezione'}, inplace=True )
    sezionetocomune['coordinate'] = sezionetocomune.sezione.apply(lambda c: TROVACOORDINATE(c,yx,UL))
    sezionetocomune['Longitudine'] = sezionetocomune.coordinate.apply(lambda l: l[1])
    sezionetocomune.coordinate = sezionetocomune.coordinate.apply(lambda u: u[0])
    sezionetocomune.rename(columns = {'coordinate' : 'Latitudine'}, inplace= True)
    sezionetocomune.comune.fillna('Mare', inplace= True)

    congeometry = gpd.GeoDataFrame(sezionetocomune, geometry=gpd.points_from_xy(sezionetocomune.Longitudine, sezionetocomune.Latitudine))
    congeometry = congeometry[['sezione', 'comune', 'geometry']]
    # congeometry.to_file(f'{cartella_dove_printare}/SHP/sezione2comuniSHP.shp')
    return print(congeometry.head())



# def run(area, start, end, mini, minj, nrows, ncols, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare):
#
#         sezionetocomune.comune.fillna( 'Mare', inplace=True)
#         delta = datetime.timedelta(hours= 1)
#         cal = start
#         # print(f'ram iniziale in mb: {psutil.virtual_memory().used/1000000}') ???????????
#
#         while cal <= end:
#             time = data_amg(cal)#%Y%m%d
#             giorno = f'{cartella_dei_dati}/{area}/{time}/'    #dove leggiamo
#
#             cnt = 0
#             while cnt < 96:
#                 # hb = pd.DataFrame(np.zeros((pd.to_numeric(header.loc['NROWS', 'value']), pd.to_numeric(header.loc['NCOLS', 'value']))))
#                 hb = BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt)
#                 print(f'ho letto {cnt}.bil')
#                 cnt += 1
#                 for f in range(3):
#                     hb = hb.merge(BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt))
#                     print(f'ho sommato {cnt}.bil')
#                     cnt += 1
#                 ToCSVConComune(hb/4, nrows, ncols, mini, minj, time, data_amg_om(cal), sezionetocomune, cartella_dove_printare, city, end) #crea la matrice di numeri e la legge q (che sta per quarter)
#                 cal += delta
#                 # break
#
# def ToCSVConComune(hb, nrows, ncols, mini, minj, time, ora, sezionetocomune, cartella_dove_printare, city, end):
#     d = {'comune': sezionetocomune.comune.unique(), 'val': 0}
#     df = pd.DataFrame(data = d)
#     for i in range(nrows):
#         for j in range(ncols):
#             sezione = f'{mini + i}-{minj + j}'  #indice di riga e colonna dal minimo mi muovo degli indici
#             valore = pd.to_numeric(hb.iloc[(mini + i), (minj + j)])
#             com = sezionetocomune.loc[sezione,"comune"]
#             # out.write(f'{data_amg_om(start)},{sezione},{com},{valore}\n')
#             df.loc[df.comune == com, 'val'] += valore
#
#     # ru = psutil.virtual_memory().used
#     df['ora'] = ora
#     df.to_csv(f'{cartella_dove_printare}/{city.name}_{time}_{data_amg(end)}_24.csv', mode = 'a', header = None, index = False)
#
#     return print(f'Fatto') #delta ram mb: {ru/1000000}


def TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune):

    hb.set_index(hb['riga-colonna'].unique(),inplace = True)
    hb = hb.drop(columns = 'riga-colonna')
    for i in hb.columns:
        hb[i].mask((hb[i]>= 65535), inplace = True)
    hb['value'] = hb.mean(axis = 1)

    #hb = hb['value'].reset_index()
    # sezionetocomune.reset_index(inplace = True)
    # hb.rename(columns = {'index': 'sezione'}, inplace = True)
    # sezionetocomune.rename(columns = {'index': 'sezione'}, inplace = True)
    # hb = pd.DataFrame(hb).join(pd.DataFrame(sezionetocomune))# tiene tutte le sezioni passando comune nan dove non è definito
    hb = pd.DataFrame(hb).merge(pd.DataFrame(sezionetocomune),left_index=True, right_index=True) #tiene solo le righe che hanno comune
    hb.comune.fillna('Mare', inplace = True)
    hb = hb.groupby('comune').aggregate('value').sum().reset_index()
    hb['time'] = data_amg_om(cal)
    hb.to_csv(f'{cartella_dove_printare}/{city.name}_{data_amg(cal)}_{data_amg(end)}_24.csv', mode = 'a', index = False, header = None)
    print('ho fatto csv')


def run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare):

        # sezionetocomune.comune.fillna( 'Mare', inplace=True)
        delta = datetime.timedelta(hours= 1)
        cal = start
        # print(f'ram iniziale in mb: {psutil.virtual_memory().used/1000000}') ???????????

        while cal <= end:
            time = data_amg(cal)#%Y%m%d
            giorno = f'{cartella_dei_dati}/{city.name}/{time}/'    #dove leggiamo

            cnt = 0
            while (cnt < 96) & (cal <= end):
                # hb = pd.DataFrame(np.zeros((pd.to_numeric(header.loc['NROWS', 'value']), pd.to_numeric(header.loc['NCOLS', 'value']))))
                hb = BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt)
                print(f'ho letto {cnt}.bil')
                cnt += 1
                for f in range(3):
                    hb = hb.merge(BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt))
                    print(f'ho sommato {cnt}.bil')
                    cnt += 1
                TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune)
                cal += delta
                # break

def MAPPASEZIONECOMUNI (sezioni, marche, cartella_dove_printare, densità_puntini):
    sezioni = sezioni.rename(columns = {'comune' : 'COMUNE'})
    mappa = folium.Map(location=[marche.centroid.y.mean(), marche.centroid.x.mean()], tiles="cartodbpositron", zoom_start=8)
    #  = sezioni.head(8000)
    comuni = {}
    for i in range(len(sezioni)):
        if i % densità_puntini == 0 :
            if i % 10000 == 0: print(f'siamo arrivati: {i}')
            if sezioni.iloc[i,1] in comuni:
                folium.Circle(
                radius=75,
                location=[pd.to_numeric(sezioni.iloc[i,2])[1], pd.to_numeric(sezioni.iloc[i,2])[0]],
                color= comuni.get(sezioni.iloc[i,1]),
                fill=True,
                fill_color = comuni.get(sezioni.iloc[i,1]),
                fill_opacity = 1.0,
                ).add_to(mappa)
            else:
                comuni[sezioni.iloc[i,1]] = '#'+''.join(random.choice('ABCDEF1234567890') for k in range(6))
                folium.Circle(
                radius=75,
                location= [pd.to_numeric(sezioni.iloc[i,2])[1], pd.to_numeric(sezioni.iloc[i,2])[0]],
                color= comuni.get(sezioni.iloc[i,1]),
                fill=True,
                fill_color = comuni.get(sezioni.iloc[i,1]),
                fill_opacity = 1.0,
                ).add_to(mappa)
    # folium.LayerControl().add_to(mappa)
    return mappa

def CREACARTINA(percorso_file, cartella_dove_printare):
    map = gpd.read_file(percorso_file)
    # map.loc[map['COMUNE'] == 'Carpi'].plot()
    # map.plot()
    # map.loc[pd.to_numeric(map['COD_REG']) == 20].plot()
    map = map.loc[pd.to_numeric(map['COD_REG']) > 7]
    map = map.loc[pd.to_numeric(map['COD_REG']) < 14]
    map = map[['COMUNE','geometry','COD_REG']]
    map.to_file(f'{cartella_dove_printare}/SHP/Centroitalia/centroitaliaSHP.shp')
    return map
#DATI
cartella_dei_dati = f'D:/Università/Tirocinio/Analisi Dati/Dati Grezzi'
cartella_dove_printare = f'D:/Università/Tirocinio/Analisi Dati/Grezzo Niki'
area = "CentroItalia"
coordinates = [13.001457, 42.992867, 13.941032, 43.661072]
city = City(area, coordinates)
UL = [43.7672915262, 11.8341670519]  #Upper Left Y,X Map
yx = pd.Series([0.0012500001, 0.0016666668])  #Dimensione y,x sezione
month = '04' #mese
year = '2015' #anno
start = datetime.datetime(int(year), int(month), 16, 0, 0, 0)
end = datetime.datetime(int(year), int(month), 16, 0, 50, 0)

marche = gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Dati/shp/Comuni2016/marche.shp')
# sezioni = gpd.read_file(f'{cartella_dove_printare}/SHP/sezione2comuneSHP.shp')
header = READHDR(f'{cartella_dei_dati}/{area}/20150401/0.hdr')
df_util = DF_UTIL(header,city)
# minj =df_util.loc['minj','valori']
# mini =df_util.loc['mini','valori']
# nrows =df_util.loc['nrows','valori']
# ncols =df_util.loc['ncols','valori']
sezionetocomune = pd.read_csv(f'{cartella_dove_printare}/sezione2comuneDEF.csv',index_col= 0)




## Esecuzione di Sezione2Comuni.csv

In [143]:
mappa = CREACARTINA('D:/Università/Tirocinio/Analisi Dati/Dati/shp/Comuni2016/comuni2016.shp',cartella_dove_printare)
SEZIONI2COMUNI(header, cartella_dove_printare, mappa )

PUNTISEZIONE(sezionetocomune, cartella_dove_printare)
run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare)

map = MAPPASEZIONECOMUNI (sezioni, marche, cartella_dove_printare, 7)
map







# areas = ["CentroItalia","Emilia","Emilia"]
# cities = [City("MarcheQuakeArea", [13.001457, 42.992867, 13.941032, 43.661072]),City("MoQuakeArea", [10.519588, 44.454968, 11.761396, 44.951228]),City("BolognaRavennaCesena", [11.221439,44.101773,12.290967,44.418104])]


sono alla riga: 0
sono alla riga: 200


KeyboardInterrupt: 

UL = [43.7672915262, 11.8341670519]  #Upper Left Y,X Map
yx = pd.Series([0.0012500001, 0.0016666668])  #Dimensione y,x sezione
map = gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Dati/shp/Comuni2016/marche.shp')
comuni = map[['COMUNE', 'geometry']]

# Il primo numero della chiave è l'indice di riga nel reticolo a disposizione, il secondo numero è l'indice di colonna
df['coordinate'] = df[1].apply(lambda c: TROVACOORDINATE(c,yx,UL))
df = df[[1, 'coordinate']]
# y = latitudine, x = longitudine
df['Longitudine'] = df['coordinate'].apply(lambda k: k[1])
df['coordinate'] = df['coordinate'].apply(lambda k: k[0])
df = df.rename(columns = {'coordinate': 'Latitudine',1: 'chiave'})
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitudine, df.latitudine))
gdf = gdf[['chiave', 'geometry']]
gdf['comuni'] = gdf['geometry'].apply(lambda punto: QUADRATINO2COMUNE(punto, comuni))
gdf = gdf[['chiave', 'comuni']]
gdf.to_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/Sezioni2Comuni.csv',index= False)

##       metodo di associazione differente rispetto a quello adottato in run


ran = 12
delta = datetime.timedelta(days=1)
while start < end:
    time = data_amg(start)  #%Y%m%d
    br = pd.read_csv(f'D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/prova1{area}_{time}_{data_amg(end)}.csv',
                     header=None)
    br = br.rename(columns={0: 'time', 1: 'cella', 2: 'val'})
    br['time'] = br['time'].apply(lambda u: f'{u[:-2]}00')
    br['val'] = br['val'].astype(float)
    agg = br.groupby(['time', 'cella']).aggregate('val').mean().reset_index()
    agg.to_csv(
        'D:/Università/Tirocinio/Analisi Dati/Grezzo Niki' + '/' + city.name + '_' + data_amg(start) + '_' + data_amg(
            end) + '_24.csv', index=False)
    start += delta

## associazione comune a sezione
produzione di Sezione2comuni.csv

UL = [43.7672915262, 11.8341670519]  #Upper Left Y,X Map
yx = pd.Series([0.0012500001, 0.0016666668])  #Dimensione y,x sezione
map = gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Dati/shp/Comuni2016/marche.shp')
comuni = map[['COMUNE', 'geometry']]
df = pd.read_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/prova_20150416.csv', header=None)
# Il primo numero della chiave è l'indice di riga nel reticolo a disposizione, il secondo numero è l'indice di colonna
df['coordinate'] = df[1].apply(lambda c: TROVACOORDINATE(c,yx,UL))
df = df[[1, 'coordinate']]
# y = latitudine, x = longitudine
df['Longitudine'] = df['coordinate'].apply(lambda k: k[1])
df['coordinate'] = df['coordinate'].apply(lambda k: k[0])
df = df.rename(columns = {'coordinate': 'Latitudine',1: 'chiave'})
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitudine, df.latitudine))
gdf = gdf[['chiave', 'geometry']]
gdf['comuni'] = gdf['geometry'].apply(lambda punto: QUADRATINO2COMUNE(punto, comuni))
gdf = gdf[['chiave', 'comuni']]
gdf.to_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/Sezioni2Comuni.csv',index= False)

## aggiunta della colonna comuni ai csv coi val

vals = pd.read_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/CentroItalia_20150416_20150416_24palla.csv')
sc = pd.read_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/Sezioni2Comuni.csv')
vals = vals.rename(columns = {'cella' : 'chiave'})
vals['chiave']= vals['chiave'].astype(str)
sc['chiave']= sc['chiave'].astype(str)
vals['chiave'] = vals['chiave'].apply(lambda d: d.replace(' ', ''))
sc['chiave'] = sc['chiave'].apply(lambda d: d.replace(' ', ''))
mergiato = vals.merge(sc)
mergiato.sort_values('comuni')
mergiato.comuni = mergiato.comuni.fillna('MARE')


In [ ]:
## Creazione del shp con nome sezione e Comune


In [38]:
# base = pd.read_csv(cartella_dove_printare + '/CentroItalia_20150416_20150416_OK.csv',header= None)
# base.rename(columns = {0 : 'ora', 1: 'sezione', 2 :'comune',3: 'val' }, inplace = True)
# gruppato = base.groupby(['ora','comune']).aggregate('val').sum().reset_index()
# gruppato.to_csv(cartella_dove_printare + '/gruppato.csv',index = False)

In [26]:
# def ToCSVConComune(hb, nrows, ncols, mini, minj, time, start, sezionetocomune, cartella_dove_printare, city, end):
#     df = pd.DataFrame(columns = ['ora', 'val'], index = sezionetocomune.comune.unique())
#     df.ora = start
#     for i in range(nrows):
#         for j in range(ncols):
#             sezione = f'{mini + i}-{minj + j}'  #indice di riga e colonna dal minimo mi muovo degli indici
#             valore = hb.iloc[(mini + i), (minj + j)]
#             com = sezionetocomune.loc[sezione,"comune"]
#             # out.write(f'{data_amg_om(start)},{sezione},{com},{valore}\n')
#             df.loc[com, 'val'] += valore
#     df.to_csv(f'{cartella_dove_printare}/{city.name}_{time}_{data_amg(end)}_OK.csv', mode = 'a', header = None)
#
#     return print('fatto')


## creazione file shp delle regioni unite

In [ ]:
# map = gpd.read_file(f'{dir}/Comuni/Limiti01012016/Com01012016/Com01012016_WGS84.shp')
# print(map)
# map.loc[map['COMUNE'] == 'Formigine'].plot()
# map.plot()
# map.loc[pd.to_numeric(map['COD_REG']) == 20].plot()
# map = map.loc[pd.to_numeric(map['COD_REG']) > 8]
# map = map.loc[pd.to_numeric(map['COD_REG']) < 18]
# map.plot()
# map.to_file('C:/Users/giord/Desktop/Comuni_centro_Italia.shp')
# map2 = gpd.read_file('C:/Users/giord/Desktop/Comuni_centro_Italia.shp')
# map2.plot()

In [11]:
import time
def BILREADER_CORRETTO(filename, header,cnt):
    nrows =np.arange(int(header.iloc[3,0]))
    ncols = np.arange(int(header.iloc[4,0]))
    x = np.array(np.meshgrid(nrows,ncols )).transpose().reshape((int(header.iloc[3,0])*int(header.iloc[4,0]),2))
    np.array2string(x,separator='-')[1:-1].replace(' ','')


    # np.fromfile(filename, dtype='<u2', count=ncols, sep="")
         # cellecentocinquanta = cellecentocinquanta.append(riga, ignore_index=True)
    # cellecentocinquanta = cellecentocinquanta.head(50)
    # cellecentocinquanta
    # cellecentocinquanta = cellecentocinquanta.reset_index().melt(id_vars= ['index'], value_name= str(cnt))
    # cellecentocinquanta['riga-colonna'] = cellecentocinquanta['index'].astype(str) +'-'+cellecentocinquanta.variable.astype(str)
    # cellecentocinquanta= cellecentocinquanta[['riga-colonna', str(cnt)]]
    return np.fromfile(filename, dtype='<u2', count=ncols, sep="")


cnt = 0
# start = time.time()
hb = BILREADER_CORRETTO('D:/Università/Tirocinio/Analisi Dati/Dati Grezzi/CentroItalia/20150401/0.bil', header,cnt)
# end = time.time()

np.array(np.fromfile('D:/Università/Tirocinio/Analisi Dati/Dati Grezzi/CentroItalia/20150401/0.bil', dtype='<u2', sep="")).shape


TypeError: only integer scalar arrays can be converted to a scalar index

In [42]:
nrows =np.arange(int(header.iloc[3,0]))
ncols = np.arange(int(header.iloc[4,0]))
x = np.array(np.meshgrid(nrows,ncols )).transpose().reshape((int(header.iloc[3,0])*int(header.iloc[4,0]),2))
y = np.array(str)
for i in range(len(x)):
   np.array2string(x[i],separator='-')[1:-1].replace(' ','')


()

In [13]:
def TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune):

    hb.set_index(hb['riga-colonna'].unique(),inplace = True)
    hb = hb.drop(columns = 'riga-colonna')
    for i in hb.columns:
        hb[i].mask((hb[i]>= 65535), inplace = True)
    hb['value'] = hb.mean(axis = 1)
    #hb = hb['value'].reset_index()
    # sezionetocomune.reset_index(inplace = True)
    # hb.rename(columns = {'index': 'sezione'}, inplace = True)
    # sezionetocomune.rename(columns = {'index': 'sezione'}, inplace = True)
    # hb = pd.DataFrame(hb).join(pd.DataFrame(sezionetocomune))# tiene tutte le sezioni passando comune nan dove non è definito
    hb = pd.DataFrame(hb).merge(pd.DataFrame(sezionetocomune),left_index=True, right_index=True) #tiene solo le righe che hanno comune
    print(hb.head())

    hb.comune.fillna('Mare', inplace = True)
    hb = hb.groupby('comune').aggregate('value').sum().reset_index()
    hb['time'] = data_amg_om(cal)
    hb.to_csv(f'{cartella_dove_printare}/{city.name}_{data_amg(cal)}_{data_amg(end)}_24.csv', mode = 'a', index = False, header = None)
    print('ho fatto csv')


def run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare):
        print('inizio Run')
        delta = datetime.timedelta(hours= 1)
        cal = start
        # print(f'ram iniziale in mb: {psutil.virtual_memory().used/1000000}') ???????????

        while cal <= end:
            print(f'processo {cal.date()}')
            data = data_amg(cal)#%Y%m%d
            giorno = f'{cartella_dei_dati}/{city.name}/{data}/'    #dove leggiamo

            cnt = 0
            while (cnt < 96) & (cal <= end):
                # hb = pd.DataFrame(np.zeros((pd.to_numeric(header.loc['NROWS', 'value']), pd.to_numeric(header.loc['NCOLS', 'value']))))
                inizio = time.time()
                hb = BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt)
                print(f'fatto {cnt}.bil')
                # print(f'ho letto {cnt}.bil')
                cnt += 1
                for f in range(3):
                    hb = hb.merge(BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt))
                    print(f'fatto {cnt}.bil')
                    cnt += 1
                TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune)
                cal += delta
                print(f'Siamo a {cal.time()}. Per questo csv ho impiegato: {time.time() - inizio}')
            break

run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare)

inizio Run
processo 2015-04-16
fatto 0.bil
fatto 1.bil
fatto 2.bil
fatto 3.bil
      0   1   2   3  value    comune
0-0   2   2   2   2    2.0  Bibbiena
1-0   4   4   4   4    4.0  Bibbiena
2-0   7   7   7   7    7.0  Bibbiena
3-0   9   9   9   9    9.0  Bibbiena
4-0  10  10  10  10   10.0  Bibbiena
ho fatto csv
Siamo a 01:00:00. Per questo csv ho impiegato: 49.05515933036804
